<a href="https://colab.research.google.com/github/Ayancodes2601/AudioTranscriber/blob/main/Audio_Transcriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-78ipmktx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-78ipmktx
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798395 sha256=8e97052230f3de87e1478729fe5a8fb9bc1039d1afae6d2f7f326fa92eca0396
  Stored in directory: /tmp/pip-ephem-wheel-cache-joo8iy62/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://dev

In [ ]:
!whisper "/content/drive/MyDrive/Lecture 2 _ Image Classification.mp4" --model medium.en

100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 130MiB/s]
[00:00.000 --> 00:02.840]  Welcome to lecture two of CS231n.
[00:02.840 --> 00:05.400]  On Tuesday, we, just recall, we sort of gave you
[00:05.400 --> 00:07.800]  the big picture view of what is computer vision,
[00:07.800 --> 00:09.840]  what is the history, and a little bit of the overview
[00:09.840 --> 00:12.240]  of the class, and today, we're really gonna dive in
[00:12.240 --> 00:14.040]  for the first time into the details,
[00:14.040 --> 00:16.080]  and we'll start to see in much more depth
[00:16.080 --> 00:18.440]  exactly how some of these learning algorithms
[00:18.440 --> 00:20.440]  actually work in practice.
[00:20.440 --> 00:22.520]  So the first lecture of the class is going to be
[00:22.520 --> 00:24.840]  a little bit more detailed, and we'll start to see
[00:24.840 --> 00:28.000]  how these learning algorithms actually work in practice.
[00:28.040 --> 00:29.720]  So the first lecture 

In [ ]:
import re
import json

def group_complete_thoughts(timestamps):
    grouped_timestamps = []
    current_group = []

    for start_time, text in timestamps:
        current_group.append(text)

        if text[-1] in ['.', '!', '?'] and len(current_group) >= 3:
            grouped_timestamps.append((start_time, " ".join(current_group)))
            current_group = []

    return grouped_timestamps

def format_time(time_str):
    h, m, s = map(int, re.sub(r',\d{3}', '', time_str).split(':'))
    return f"{h:02d}:{m:02d}:{s:02d}"

def srt_to_json(srt_content):
    json_data = {
        "dataset_link": "https://www.youtube.com/watch?v=OoUX-nOEjG0&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=2",
        "dataset": {}
    }

    pattern = r"\d+\n(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\n(.+?)(?=\n\d+\n\d{2}:\d{2}:\d{2},\d{3} -->|\Z)"
    matches = re.findall(pattern, srt_content, re.DOTALL)

    timestamps = [(format_time(match[0]), match[2].strip()) for match in matches]
    grouped_timestamps = group_complete_thoughts(timestamps)

    for i, (start_time, combined_text) in enumerate(grouped_timestamps):
        json_data["dataset"][start_time] = combined_text

    return json.dumps(json_data, indent=2)

# Replace this with the actual path to your SRT file
srt_file_path = "/content/Lecture 2 _ Image Classification.srt"
with open(srt_file_path, "r") as srt_file:
    srt_content = srt_file.read()
    json_data = srt_to_json(srt_content)

with open("transcript1.json", "w") as file:
    file.write(json_data)
